In [1]:
### Import packages ###
import numpy as np
import pandas as pd
import itertools

# Input Data Set 

In [2]:
### Regression ###
# Data =["Auto"]
# Data =["BostonHousing"]
# Data =["CPS"]
# Data =["Simulate"]
# Data =["Yacht"]

### Classification ###
Data =["BostonHousingBinned"]


# Input directory

In [3]:
# Directory
dir_path = "/Users/simondn/Documents/RashomonActiveLearning/Code/raw"

## Input Rashomon Parameters

In [4]:
# Rashomon QBC parameters
Seed = range(0,10)
TestProportion = [0.2]
CandidateProportion = [0.8]
SelectorType = ["RashomonQBCFunction", "PassiveLearning"]
ModelType = ["TreeFarmsFunction"]
DataArgs = [{"N": 1000, "K": 5}]
SelectorArgs = [{"TopCModels": 1}, {"TopCModels": 10}, {"TopCModels": 100}]
ModelArgs = [{"TopCModels": 1, "config" : {"regularization": 0.01, "rashomon_bound_multiplier": 0.15}, "Type":"Classification"},
             {"TopCModels": 10, "config" : {"regularization": 0.01, "rashomon_bound_multiplier": 0.15}, "Type":"Classification"},
             {"TopCModels": 100, "config" : {"regularization": 0.01, "rashomon_bound_multiplier": 0.15}, "Type":"Classification"}]

filtered_combinations = [
    (seed, data, test_prop, cand_prop, selector_type, model_type, data_arg, sel_arg, mod_arg)
    for seed, data, test_prop, cand_prop, selector_type, model_type, data_arg, sel_arg, mod_arg in itertools.product(
        Seed, Data, TestProportion, CandidateProportion, SelectorType, ModelType, DataArgs, SelectorArgs, ModelArgs
    )
    if sel_arg["TopCModels"] == mod_arg["TopCModels"] and 
       (selector_type != "PassiveLearning" or sel_arg["TopCModels"] == 1)
]

df = pd.DataFrame(filtered_combinations, columns=[
    "Seed", "Data", "TestProportion", "CandidateProportion", "SelectorType", 
    "ModelType", "DataArgs", "SelectorArgs", "ModelArgs"])

### Rashomon numbering ###
df["RashomonNumbering"] = df['SelectorArgs'].apply(lambda x: str(x['TopCModels']))


# Input Non-Rashomon parameters

In [5]:
# # Regression parameters
# Seed = range(1,3)
# TestProportion = [0.2]
# CandidateProportion = [0.8]
# SelectorType = ["PassiveLearning", "GSxFunction", "GSyFunction", "iGSFunction"]
# ModelType = ["LinearRegressionFunction", "RidgeRegressionFunction", "RandomForestRegressorFunction"]
# DataArgs = [{"N": 1000, "K": 5}]
# SelectorArgs = [{"df_Candidate" : "df_Candidate"}]
# ModelArgs = [{"alpha_val": 0.1,
#               "n_estimators" :100,
#               "Seed" : 1}]

# ### Combinations ###
# Combinations = list(itertools.product(
#     Seed, Data, TestProportion, CandidateProportion, SelectorType, ModelType, DataArgs, SelectorArgs, ModelArgs
# ))

# ### Data Frame ###
# df = pd.DataFrame(Combinations,
#                   columns = ["Seed", 
#                   "Data", 
#                   "TestProportion", 
#                   "CandidateProportion", 
#                   "SelectorType", 
#                   "ModelType", 
#                   "DataArgs", 
#                   "SelectorArgs", 
#                   "ModelArgs"])

In [6]:
### Job Name ###
ReplacementCharacters = {":": "",  " ": "", "{": "", "}": "", """: "", ",": "_", """: "", "[": "", "]": ""}
DataArgsReplacement = "".join([ReplacementCharacters.get(char, char) for char in str(DataArgs)])
ModelArgsReplacement = "".join([ReplacementCharacters.get(char, char) for char in str(ModelArgs)])
SelectorArgsReplacement = "".join([ReplacementCharacters.get(char, char) for char in str(ModelArgs)])
ModelTypeReplacement = str(ModelType).replace("Function","")
DataReplacement = "".join([ReplacementCharacters.get(char, char) for char in str(Data)])

df["JobName"] = ("Seed" + df["Seed"].astype(str) +
                 "_Data" + df["Data"].astype(str) +
                 "_TP" + df["TestProportion"].astype(str) +
                 "_CP" + df["CandidateProportion"].astype(str) +
                 "_ST" + df["SelectorType"].astype(str) +
                 "_MT" + df["ModelType"].astype(str) + 
                 "RashomonNum" + df["RashomonNumbering"]
                #  DataArgsReplacement + "_" +
                #  ModelArgsReplacement
                 )


In [7]:
### Output Name ###
df["JobName"] = df["JobName"].str.replace("Function", "", regex=False)
df["Output"] =  df["Data"].astype(str) + "/" + df["ModelType"].astype(str) + "/Raw/" + df["JobName"]+ df["RashomonNumbering"] + ".pkl"
df["Output"] = df["Output"].str.replace("Function", "", regex=False)
df.drop('RashomonNumbering', axis=1, inplace=True)


In [8]:
# Define the columns to process
columns_to_process = ['DataArgs', 'SelectorArgs', 'ModelArgs']
for col in columns_to_process:
    df[col] = "'" + df[col].apply(lambda x: str(x).replace("'", '"') if pd.notna(x) else x) + "'"


In [9]:
### Save Parameter Vector ###
output_path = "/Users/simondn/Documents/RashomonActiveLearning/Data/ParameterVectors/ParameterVector" + str(Data[0]) + ".csv"
df.to_csv(output_path, index=False)